In [3]:
import glob 
import zipfile
import numpy as np
import cv2
import os
import time

from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models

import pandas as pd
from sklearn.model_selection import train_test_split


data_root_path = 'drive/My Drive/colab_data/dogs-vs-cats/numpy_folder/'

train_data = []     
def read_data_from_npy_zip():
    for f in tqdm(os.listdir(data_root_path)):
      path = os.path.join(data_root_path, f)
      train_data.extend(np.load(path, allow_pickle=True))

read_data_from_npy_zip()

np.random.shuffle(train_data)
print(len(train_data), train_data[0])

100%|██████████| 26/26 [01:08<00:00,  2.65s/it]

25000 [array([[[ 59,  50,  63],
        [ 54,  47,  60],
        [ 56,  53,  63],
        ...,
        [112, 100,  95],
        [136, 122, 114],
        [137, 115, 110]],

       [[ 65,  56,  69],
        [ 57,  50,  63],
        [ 54,  50,  61],
        ...,
        [ 88,  76,  70],
        [127, 114, 104],
        [146, 129, 118]],

       [[ 73,  63,  76],
        [ 63,  57,  68],
        [ 54,  50,  61],
        ...,
        [ 73,  62,  56],
        [128, 118, 107],
        [139, 128, 112]],

       ...,

       [[ 53,  49,  55],
        [ 49,  47,  53],
        [ 55,  53,  59],
        ...,
        [ 86,  85,  87],
        [ 90,  87,  89],
        [ 93,  90,  90]],

       [[ 44,  40,  46],
        [ 47,  45,  51],
        [ 51,  49,  55],
        ...,
        [ 93,  92,  94],
        [ 95,  92,  94],
        [ 98,  96,  96]],

       [[ 44,  40,  46],
        [ 50,  48,  54],
        [ 53,  51,  57],
        ...,
        [ 88,  87,  89],
        [ 93,  90,  92],
        [ 95,  93

In [9]:
print(len(train_data))
print(train_data[0][0].shape)

25000
(224, 224, 3)


In [6]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

IMAGE_SIZE = (224, 224)

net = models.resnet18(pretrained=True).to(device)
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
net.fc = nn.Linear(net.fc.in_features, 2).to(device)


def run(X, y, is_training=True):
    
    optimizer = optim.Adam(net.parameters(), lr=0.001)
    loss_function = nn.MSELoss()
    
    if is_training:
        net.zero_grad()
        optimizer.zero_grad()
        
    outputs = net(X)
    matches  = [torch.argmax(i)==torch.argmax(j) for i, j in zip(outputs, y)]
    acc = matches.count(True)/len(matches)
    loss = loss_function(outputs, y)

    if is_training:
        loss.backward()
        optimizer.step()

    return acc, loss

def batch_and_run(t_data, is_training=True):
    BATCH_SIZE = 64
    for i in tqdm(range(0, len(t_data), BATCH_SIZE)):
        batch = t_data[i:i+BATCH_SIZE]

        print(len(batch))
        print(len(batch[0]))

        batch_X = torch.Tensor([np.moveaxis(x[0], -1, 0) for x in batch])
        batch_X = batch_X/255.0
        batch_y = torch.Tensor([x[1] for x in batch])

        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)

        return run(batch_X, batch_y, is_training)


def train(net):
    EPOCHS = 20  
    for epoch in range(EPOCHS):
        acc, loss = batch_and_run(train_data, True)
        #val_acc, val_loss = batch_and_run(X_test, y_test, False)
        print(f"Epoch: {epoch}, {round(time.time(),3)},{round(float(acc),2)},{round(float(loss), 4)}\n")
        
train(net)


predictions=[]
id_line=[]
def test(net):
    for f in os.listdir(test_path):
        id_line.append(f.split('.')[0])
        path = os.path.join(test_path, f)
        img=cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img=cv2.resize(img, (50, 50))
        test_X = torch.Tensor(img).view(-1, IMAGE_SIZE)
        test_X = test_X.to(device)
        net_out = net(test_X)
        
        predictions.append(torch.argmax(net_out))

test(net)

predicted_val = [x.tolist() for x in predictions]
print(predicted_val)

Running on the GPU


  0%|          | 0/391 [00:00<?, ?it/s]

64
2


  0%|          | 0/391 [00:00<?, ?it/s]

Epoch: 0, 1589704931.822,0.52,1.3793

64
2


...

  0%|          | 0/391 [00:00<?, ?it/s]

Epoch: 18, 1589704957.819,0.98,0.0677

64
2


  0%|          | 0/391 [00:01<?, ?it/s]


Epoch: 19, 1589704959.262,1.0,0.1554



NameError: ignored